In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import ParameterGrid
import time
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+git://github.com/rasbt/mlxtend.git

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
from mlxtend.frequent_patterns import association_rules

Getting data

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/лаб5/Var_4_Market_Basket_Analysis_1.csv')
d = np.array(data.replace(np.nan, 'nAAn'))
te = TransactionEncoder()
te_ary = te.fit(d).transform(d)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.drop('nAAn', axis=1, inplace=True)
df

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,bramble,brownies,bug spray,burger sauce,burgers,butter,cake,candy bars,carrots,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolate bread,chutney,cider,clothes accessories,cookies,cooking oil,corn,cottage cheese,cream,dessert wine,eggplant,eggs,energy bar,energy drink,...,oil,olive oil,pancakes,parmesan cheese,pasta,pepper,pet food,pickles,protein bar,red wine,rice,salad,salmon,salt,sandwich,shallot,shampoo,shrimp,soda,soup,spaghetti,sparkling water,spinach,strawberries,strong cheese,tea,tomato juice,tomato sauce,tomatoes,toothpaste,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

Apriori

In [ ]:
def train(table, method='apriori', **param):
  start_time = time.time()
  if method=='apriori':
    freq_sets = apriori(table, min_support=param['min_support'], use_colnames=True, max_len=25)
    rules = association_rules(freq_sets, metric="confidence", min_threshold=param['min_threshold'])
    rules = rules[rules['lift']>1]
  if method=='fpgrowth':
    freq_sets = fpgrowth(table, min_support=param['min_support'], use_colnames=True, max_len=25)
    rules = association_rules(freq_sets, metric="confidence", min_threshold=param['min_threshold'])
    rules = rules[rules['lift']>1]
  t = time.time() - start_time
  return rules, freq_sets, t


In [ ]:
qual1 = []
params = {'min_support':np.linspace(0.001,0.03,num=7),
          'min_threshold':np.linspace(0.2,0.8,num=4)
          }
param = list(ParameterGrid(params))
for p in param:
  rules, freq_sets, t = train(df, **p)
  qual1.append(t)

p_best1 = {'min_support':0.001,
          'min_threshold':0.2
          }
rules1, freq_sets1, t = train(df, **p_best1)
print('best_params:', p_best1)
print('mean time:', np.array(qual1).mean())

In [ ]:
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(almonds),(burgers),0.020267,0.087200,0.005200,0.256579,2.942419,0.003433,1.227837
1,(almonds),(chocolate),0.020267,0.163867,0.006000,0.296053,1.806668,0.002679,1.187778
2,(almonds),(eggs),0.020267,0.179733,0.006533,0.322368,1.793593,0.002891,1.210491
3,(almonds),(french fries),0.020267,0.170933,0.004400,0.217105,1.270117,0.000936,1.058976
4,(almonds),(green tea),0.020267,0.132000,0.004933,0.243421,1.844099,0.002258,1.147270
...,...,...,...,...,...,...,...,...,...
9347,"(chocolate, spaghetti, frozen vegetables, milk)","(mineral water, ground beef)",0.003467,0.040933,0.001067,0.307692,7.516913,0.000925,1.385319
9348,"(chocolate, spaghetti, ground beef, milk)","(mineral water, frozen vegetables)",0.002933,0.035733,0.001067,0.363636,10.176391,0.000962,1.515276
9349,"(ground beef, chocolate, frozen vegetables, milk)","(mineral water, spaghetti)",0.002667,0.059733,0.001067,0.400000,6.696429,0.000907,1.567111
9350,"(chocolate, spaghetti, frozen vegetables, grou...","(mineral water, milk)",0.003067,0.048000,0.001067,0.347826,7.246377,0.000919,1.459733


In [ ]:
freq_sets1

,support,itemsets
0,0.020267,(almonds)
1,0.008800,(antioxydant juice)
2,0.004667,(asparagus)
3,0.033200,(avocado)
4,0.004533,(babies food)
...,...,...
6751,0.001467,"(tomatoes, mineral water, milk, spaghetti, fro..."
6752,0.001200,"(mineral water, turkey, milk, spaghetti, froze..."
6753,0.001067,"(olive oil, tomatoes, mineral water, spaghetti..."
6754,0.001067,"(pancakes, mineral water, herb & pepper, spagh..."


FP-growth

In [ ]:
qual2 = []
params = {'min_support':np.linspace(0.001,0.03,num=7),
          'min_threshold':np.linspace(0.2,0.8,num=4)
          }
param = list(ParameterGrid(params))
for p in param:
  rules, freq_sets, t = train(df, method='fpgrowth',**p)
  qual2.append(t)

p_best2 = {'min_support':0.001,
          'min_threshold':0.2
          }
rules2, freq_sets2, t = train(df,**p_best2)
print('best_params:', p_best2)
print('mean time:', np.array(qual2).mean())

best_params: {'min_support': 0.001, 'min_threshold': 0.2}
mean time: 0.2708302480833871


In [ ]:
rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(almonds),(burgers),0.020267,0.087200,0.005200,0.256579,2.942419,0.003433,1.227837
1,(almonds),(chocolate),0.020267,0.163867,0.006000,0.296053,1.806668,0.002679,1.187778
2,(almonds),(eggs),0.020267,0.179733,0.006533,0.322368,1.793593,0.002891,1.210491
3,(almonds),(french fries),0.020267,0.170933,0.004400,0.217105,1.270117,0.000936,1.058976
4,(almonds),(green tea),0.020267,0.132000,0.004933,0.243421,1.844099,0.002258,1.147270
...,...,...,...,...,...,...,...,...,...
9347,"(chocolate, spaghetti, frozen vegetables, milk)","(mineral water, ground beef)",0.003467,0.040933,0.001067,0.307692,7.516913,0.000925,1.385319
9348,"(chocolate, spaghetti, ground beef, milk)","(mineral water, frozen vegetables)",0.002933,0.035733,0.001067,0.363636,10.176391,0.000962,1.515276
9349,"(ground beef, chocolate, frozen vegetables, milk)","(mineral water, spaghetti)",0.002667,0.059733,0.001067,0.400000,6.696429,0.000907,1.567111
9350,"(chocolate, spaghetti, frozen vegetables, grou...","(mineral water, milk)",0.003067,0.048000,0.001067,0.347826,7.246377,0.000919,1.459733


In [ ]:
freq_sets2

,support,itemsets
0,0.020267,(almonds)
1,0.008800,(antioxydant juice)
2,0.004667,(asparagus)
3,0.033200,(avocado)
4,0.004533,(babies food)
...,...,...
6751,0.001467,"(tomatoes, mineral water, milk, spaghetti, fro..."
6752,0.001200,"(mineral water, turkey, milk, spaghetti, froze..."
6753,0.001067,"(olive oil, tomatoes, mineral water, spaghetti..."
6754,0.001067,"(pancakes, mineral water, herb & pepper, spagh..."


In [ ]:
np.all(freq_sets2==freq_sets1)

True

Predictions

In [ ]:
def pred(ar,X,t=True): # return consequences with the highest conf
  y = pd.DataFrame(columns=ar.columns)
  i = 0
  if t==True:
    for x in list(X['itemsets']):
      if (x in list(ar['antecedents'])):
        conf_max = ar[ar['antecedents']==x]['confidence'].max()
        y = y.append(ar[ar['confidence']==conf_max][ar['antecedents']==x].iloc[0], ignore_index=True)
  else:
    for x in list(X['itemsets']):
      if (x in list(ar['antecedents'])):
        y = y.append(ar[ar['antecedents']==x], ignore_index=True)
  return y

In [ ]:
X = np.array(df.copy())
np.random.shuffle(X)
X = X[:int(0.35*X.shape[0])]
df_test = pd.DataFrame(X,
                  columns=df.columns)
df_test

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,body spray,bramble,brownies,bug spray,burger sauce,burgers,butter,cake,candy bars,carrots,cauliflower,cereals,champagne,chicken,chili,chocolate,chocolate bread,chutney,cider,clothes accessories,cookies,cooking oil,corn,cottage cheese,cream,dessert wine,eggplant,eggs,energy bar,energy drink,...,oil,olive oil,pancakes,parmesan cheese,pasta,pepper,pet food,pickles,protein bar,red wine,rice,salad,salmon,salt,sandwich,shallot,shampoo,shrimp,soda,soup,spaghetti,sparkling water,spinach,strawberries,strong cheese,tea,tomato juice,tomato sauce,tomatoes,toothpaste,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,

Apriori

In [ ]:
X_test1 = apriori(df_test, min_support=0.01, use_colnames=True, max_len=25)
y_pred1 = pred(rules1, X_test1, t=True)
y_pred1

In [ ]:
y_pred1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(almonds),(mineral water),0.020267,0.238267,0.007467,0.368421,1.546255,0.002638,1.206078
1,(avocado),(mineral water),0.033200,0.238267,0.011467,0.345382,1.449559,0.003556,1.163629
2,(barbecue sauce),(mineral water),0.010800,0.238267,0.003867,0.358025,1.502622,0.001293,1.186546
3,(black tea),(mineral water),0.014267,0.238267,0.005333,0.373832,1.568964,0.001934,1.216500
4,(body spray),(french fries),0.011467,0.170933,0.004267,0.372093,2.176831,0.002307,1.320365
...,...,...,...,...,...,...,...,...,...
249,"(mineral water, milk, ground beef)",(spaghetti),0.011067,0.174133,0.004400,0.397590,2.283252,0.002473,1.370939
250,"(mineral water, pancakes, ground beef)",(spaghetti),0.007467,0.174133,0.003067,0.410714,2.358620,0.001766,1.401471
251,"(mineral water, spaghetti, ground beef)",(frozen vegetables),0.017067,0.095333,0.004400,0.257812,2.704327,0.002773,1.218919
252,"(mineral water, milk, spaghetti)",(chocolate),0.015733,0.163867,0.004933,0.313559,1.913503,0.002355,1.218071


In [ ]:
y_pred1['confidence'].mean()

0.37227599468914785

FP-growth

In [ ]:
X_test2 = fpgrowth(df_test, min_support=0.01, use_colnames=True, max_len=25)
y_pred2 = pred(rules2, X_test2)
y_pred2

In [ ]:
y_pred2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cake),(mineral water),0.081067,0.238267,0.027467,0.338816,1.422002,0.008151,1.152074
1,(pancakes),(mineral water),0.095067,0.238267,0.033733,0.354839,1.489250,0.011082,1.180687
2,(brownies),(mineral water),0.033733,0.238267,0.009600,0.284585,1.194397,0.001562,1.064743
3,(muffins),(french fries),0.024133,0.170933,0.005333,0.220994,1.292869,0.001208,1.064263
4,(frozen smoothie),(mineral water),0.063200,0.238267,0.020133,0.318565,1.337012,0.005075,1.117838
...,...,...,...,...,...,...,...,...,...
250,"(fresh bread, milk)",(mineral water),0.009200,0.238267,0.004000,0.434783,1.824773,0.001808,1.347682
251,"(mineral water, fresh bread)",(chocolate),0.013333,0.163867,0.004000,0.300000,1.830757,0.001815,1.194476
252,"(fresh bread, french fries)",(milk),0.008267,0.129600,0.002400,0.290323,2.240143,0.001329,1.226473
253,"(chocolate, fresh bread)",(mineral water),0.009467,0.238267,0.004000,0.422535,1.773371,0.001744,1.319099


Conclusion

Методи побудови асоціативних правил, які використовувалися в цій роботі:

Apriori - алгоритм побудови частих наборів. Спочатку відшукуємо усі одноелементні часті набори (згідно з параметром мінімальної підтримки), далі по циклу двоелементні, 3-ьохелемнтні і т.д.(до деякого одноелементного набору прієднуємо усі можливі набори з попереднього етапу циклу) допоки не отримаємо порожню множину (оскільки набори на подальших кроках вже точно будуть нечастими).

Fp-growth - алгоритм побудови частих наборів за допомогою FP-дерева та умовного FP-дерева. Кожен вузол FP-дерева буде структурою з деяким елементом та його підтримкою(причому вузли елементів з більшою підтримкою будуть вище за вузли елементів з меншою підтримкою), спочатку впорядковуємо елементи за спаданням їх підтримки, далі вводимо фіктивну вершину дерева і зчитуємо кожен запис в БД - якщо такого елемента все ще немає у дереві, то створюємо відповідний з підтримкою=1 вузол, якщо вже є, то підтримка+=1. Під час побудови частих наборів проходимось по усім вузлам FP-дерева(знизу-вверх), якщо його підтримка>supp_min, то заносимо до частих наборів, потім будуємо умовне FP-дерева за цією вершиною і вже там шукаємо часті набори об'єднуючи їх із знайденим частим набором на минулому кроці і далі по циклу.

Після пошуків частих наборів будуємо асоцівтивні правила (перебираємо часті набори-розділяємо їх на умову і наслідок і якщо дане АП задовольняє обмеженням щодо confidence, то заносимо його до множини АП, також можна перевіряти ще й інші метрики).

Отримані дані свідчать, що за часом побудови частих наборів алгоритм Fp-growth є значно ефективнішим за Apriori, при цьому оптимальні параметри (для того щоб отримані АП можна було б застосувати до випадково взятої підмножини частих наборів елементів) мають бути досить маленькими як для min_supp так і для min_confidence(однак потрібно слідкувати щоб швидкість вироблених прогнозів не занадто довгою), оскільки дана БД досить різноманітна і тому кожен набір елементів є досить рідкісним.

